In [ ]:
import librosa
import numpy as np
import os
import soundfile as sf
from tqdm import tqdm

import libtsm
from madmom.features.onsets import CNNOnsetProcessor
from madmom.features.beats import RNNBeatProcessor
from madmom.features.downbeats import RNNDownBeatProcessor
from synctoolbox.dtw.mrmsdtw import sync_via_mrmsdtw
from synctoolbox.dtw.utils import compute_optimal_chroma_shift, shift_chroma_vectors, make_path_strictly_monotonic
from synctoolbox.feature.chroma import pitch_to_chroma, quantize_chroma, quantized_chroma_to_CENS
from synctoolbox.feature.pitch import audio_to_pitch_features
from synctoolbox.feature.utils import estimate_tuning


MADMOM_Fs = 44100
MADMOM_FEATURE_RATE = 100

SYNCTOOLBOX_Fs = 22050
SYNCTOOLBOX_FEATURE_RATE = 50
STEP_WEIGHTS = np.array([1.5, 1.5, 2.0])
THRESHOLD_REC = 10 ** 6

# This should be initiated by the user. 
PIANO_FILEPATH = 'sync_test/audio/piano/BeethovenLiszt_Op021-03_Katsaris_YT.wav'
ORCH_FILEPATH = 'sync_test/audio/orchestra/Beethoven_Op021-03_MarineChamberOrchestra_IMSLP.wav'
OUT_DIR = 'sync_test/sync'

if not os.path.isdir(OUT_DIR):
    os.makedirs(OUT_DIR)

In [ ]:
def resample_signal(x_in, 
                    Fs_in, 
                    Fs_out=100, 
                    norm=True, 
                    time_max_sec=None, 
                    sigma=None):
    if sigma is not None:
        x_in = ndimage.gaussian_filter(x_in, sigma=sigma)
    
    T_coef_in = np.arange(x_in.shape[0]) / Fs_in
    time_in_max_sec = T_coef_in[-1]
    
    if time_max_sec is None:
        time_max_sec = time_in_max_sec
        
    N_out = int(np.ceil(time_max_sec*Fs_out))
    T_coef_out = np.arange(N_out) / Fs_out
    
    if T_coef_out[-1] > time_in_max_sec:
        x_in = np.append(x_in, [0])
        T_coef_in = np.append(T_coef_in, [T_coef_out[-1]])
    x_out = interp1d(T_coef_in, x_in, kind='linear')(T_coef_out)
    
    if norm:
        x_max = max(x_out)
        if x_max > 0:
            x_out = x_out / max(x_out)
            
    return x_out, Fs_out


def get_chroma_features(audio,
                        tuning_offset,
                        Fs=SYNCTOOLBOX_Fs,
                        feature_rate=SYNCTOOLBOX_FEATURE_RATE,
                        verbose=False):
    f_pitch = audio_to_pitch_features(f_audio=audio,
                                      Fs=Fs,
                                      tuning_offset=tuning_offset,
                                      feature_rate=feature_rate,
                                      verbose=verbose)
    f_chroma = pitch_to_chroma(f_pitch=f_pitch)
    f_chroma_quantized = quantize_chroma(f_chroma=f_chroma)

    return f_chroma_quantized


def get_act_function(act_processor,
                     audio,
                     feature_sequence_length,
                     orig_feature_rate=MADMOM_FEATURE_RATE,
                     target_feature_rate=SYNCTOOLBOX_FEATURE_RATE,
                     sr=MADMOM_Fs):
    
    # The original feature rate by processors is 100 fps per default.
    act_function = act_processor(audio)
    
    # For downbeat activation functions
    if act_function.ndim == 2:
        act_function = act_function[:, 1]
    
    # The output signal has to be interpolated to the 
    f_novelty, Fs_out = resample_signal(act_function, 
                                        Fs_in=orig_feature_rate, 
                                        Fs_out=target_feature_rate)
    
    if f_novelty.size < feature_sequence_length:
        # The feature sequence length of the chroma features is not the same as the novelty curve
        # due to the padding while the computation of STFT for chroma features and the differentiation in spectral flux.
        diff = feature_sequence_length - f_novelty.size
        if diff % 2 == 0:
            pad = int(diff / 2)
            f_novelty = np.concatenate((np.zeros(pad), f_novelty, np.zeros(pad)))
        else:
            pad = int(diff / 2)
            f_novelty = np.concatenate((np.zeros(pad), f_novelty, np.zeros(pad)))
            null_to_append = np.array([0])
            f_novelty = np.append(f_novelty, null_to_append)
    
    return f_novelty.reshape(1, -1)


def get_obd_act_function(audio,
                         feature_sequence_length,
                         madmom_feature_rate=MADMOM_FEATURE_RATE,
                         sync_feature_rate=SYNCTOOLBOX_FEATURE_RATE,
                         madmom_sr=MADMOM_Fs):
    
    onset_processor = CNNOnsetProcessor(fps=madmom_feature_rate, sr=madmom_sr)
    beat_processor = RNNBeatProcessor(fps=madmom_feature_rate, sr=madmom_sr)
    downbeat_processor = RNNDownBeatProcessor(fps=madmom_feature_rate, sr=madmom_sr)
    
    f_onset = get_act_function(act_processor=onset_processor,
                               audio=audio,
                               feature_sequence_length=feature_sequence_length,
                               orig_feature_rate=madmom_feature_rate,
                               target_feature_rate=sync_feature_rate,
                               sr=madmom_sr)

    f_beat = get_act_function(act_processor=beat_processor,
                              audio=audio,
                              feature_sequence_length=feature_sequence_length,
                              orig_feature_rate=madmom_feature_rate,
                              target_feature_rate=sync_feature_rate,
                              sr=madmom_sr)

    f_downbeat = get_act_function(act_processor=downbeat_processor,
                                  audio=audio,
                                  feature_sequence_length=feature_sequence_length,
                                  orig_feature_rate=madmom_feature_rate,
                                  target_feature_rate=sync_feature_rate,
                                  sr=madmom_sr)
    
    f_onset = np.reshape(f_onset, f_onset.shape[1])
    f_beat = np.reshape(f_beat, f_beat.shape[1])
    f_downbeat = np.reshape(f_downbeat, f_downbeat.shape[1])
    
    f_obd = np.array([f_onset, f_beat, f_downbeat])  
    
    return f_obd

In [ ]:
def compute_features(audio_44k,
                     madmom_sr=MADMOM_Fs,
                     sync_sr=SYNCTOOLBOX_Fs):
    
    # Resample for synchronization 
    audio_22k = librosa.resample(audio_44k, 
                                 orig_sr=MADMOM_Fs, 
                                 target_sr=SYNCTOOLBOX_Fs)
    # Estimate tuning for the chroma features
    tuning_offset = estimate_tuning(audio_22k, SYNCTOOLBOX_Fs)
    
    # Compute chroma features
    f_chroma = get_chroma_features(audio_22k, 
                                   tuning_offset)

    f_obd = get_obd_act_function(audio_44k,
                                 feature_sequence_length=f_chroma.shape[1])   

    return f_chroma, f_obd
    
# Load the orchestra audio file
orch, _ = librosa.load(ORCH_FILEPATH, sr=MADMOM_Fs)

# Load the piano audio file
piano, _ = librosa.load(PIANO_FILEPATH, sr=MADMOM_Fs)

# Compute features
orch_chroma, orch_obd = compute_features(orch)
piano_chroma, piano_obd = compute_features(piano)


In [ ]:
# Find optimal chroma shift and shift chroma vectors if needed
f_cens_1hz_orch = quantized_chroma_to_CENS(orch_chroma, 201, 50, SYNCTOOLBOX_FEATURE_RATE)[0]
f_cens_1hz_piano = quantized_chroma_to_CENS(piano_chroma, 201, 50, SYNCTOOLBOX_FEATURE_RATE)[0]
opt_chroma_shift = compute_optimal_chroma_shift(f_cens_1hz_orch, f_cens_1hz_piano)
piano_chroma = shift_chroma_vectors(piano_chroma, opt_chroma_shift)

# Synchronize with MrMsDTW
wp = sync_via_mrmsdtw(f_chroma1=orch_chroma, 
                      f_onset1=orch_obd,
                      f_chroma2=piano_chroma, 
                      f_onset2=piano_obd,
                      input_feature_rate=SYNCTOOLBOX_FEATURE_RATE,
                      step_weights=STEP_WEIGHTS, 
                      threshold_rec=THRESHOLD_REC,
                      verbose=False)
wp = make_path_strictly_monotonic(wp)

pitch_shift_for_orch = -opt_chroma_shift % 12
if pitch_shift_for_orch > 6:
    pitch_shift_for_orch -= 12

In [ ]:
orch_stereo, _ = librosa.load(ORCH_FILEPATH, sr=44100, mono=False)
piano_stereo, _ = librosa.load(PIANO_FILEPATH, sr=44100, mono=False)

In [ ]:
orch_c1 = libtsm.pitch_shift(orch_stereo[0, :], pitch_shift_for_orch * 100, order="tsm-res")  
orch_c2 = libtsm.pitch_shift(orch_stereo[1, :], pitch_shift_for_orch * 100, order="tsm-res")

time_map = wp.T / SYNCTOOLBOX_FEATURE_RATE * 44100
time_map[time_map[:, 0] > len(orch_c1), 0] = len(orch_c1) - 1 
time_map[time_map[:, 1] > len(piano_stereo[0, :]), 1] = len(piano_stereo[0, :]) - 1

# Apply Time-Scale Modification
orch_c1 = libtsm.hps_tsm(orch_c1, time_map)
orch_c2 = libtsm.hps_tsm(orch_c2, time_map)

In [ ]:
# Write to the output directory
sf.write(os.path.join(OUT_DIR, 'piano.wav'), piano.T, 44100)
sf.write(os.path.join(OUT_DIR, 'orchestra.wav'), np.hstack([orch_c1, orch_c2]), 44100)